# train_chatbot

In [ ]:
# imports

# used in Tensorflow Model
import numpy as np
import tensorflow as tf
import tflearn
import random

# used to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer

# initialize LancasterStemmer
stemmer = LancasterStemmer()

# other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Download NLTK data
nltk.download('punkt')

# Load intents data

In [ ]:
print("Processing the Intents.....")
with open('intents.json') as json_data:
    intents = json.load(json_data)

# Extract information from intents

In [ ]:
words = []
classes = []
documents = []
ignore_words = ['?']

print(intents)
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")

for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        
        # add to our words list
        words.extend(w)
        documents.append((w, intent['tag']))
        
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Stem, lower, and remove duplicates

In [ ]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print(words)
print (len(documents), "documents\n")
print (len(classes), "classes\n", classes)
print (len(words), "unique stemmed words\n", words)

# Create training data

In [ ]:
print("Creating the Data for our Model.....")
training = []
output = []

print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    bag = [] # initialize our bag of words
    pattern_words = doc[0] # list of tokenized words for the pattern
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words] # stem each word

    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [ ]:
# Shuffle and convert to numpy array

print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training)

print("Creating Train and Test Lists.....")
train_x = list(training[:, 0])
train_y = list(training[:, 1])

# Build neural network

In [ ]:
print("Building Neural Network for Out Chatbot to be Contextual....")
print("Resetting graph data....")
# tf.reset_default_graph()
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Train the model

In [ ]:
print("Training....")
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

print("Training the Model.......")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)

# Save the model

In [ ]:
print("Saving the Model.......")
model.save('model.tflearn')

# Save the training data

print("Pickle is also Saved..........")
pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, open("training_data", "wb"))